In [5]:
import ee

ee.Initialize()

vatican = ee.Geometry.Point([12.4534, 41.9033]).buffer(100)

image = (
    ee.ImageCollection("COPERNICUS/S2")
    .filterBounds(vatican)
    .filterDate("2023-01-01", "2023-12-31")
    .first()
)
image.id().getInfo()

'20230105T100319_20230105T100317_T32TQM'

In [1]:
import ee

ee.initialize()

AttributeError: module 'ee' has no attribute 'initialize'

In [ ]:
collection = ee.ImageCollection("COPERNICUS/S2_SR")
first_100 = collection.toList(100)